# Production Data Loading

 1. Access _ne1_ Couchbase Database
 1. Load CSV files and convert into JSON structure 
 1. Store the JSON data in Couchbase 

## Access _ne1_ Couchbase Database

  * Create new Couchbase _bucket_ called _n1_ using the Couchbase web browser interface

In [15]:
from couchbase.bucket import Bucket
cb = Bucket('couchbase://localhost/geotechnical')  # If no errors then the access works

## Load CSV Data

In [20]:
# modified JK 2017.06.22

import json

from GeopyUtils import CsvDataReader

projects_cdr = CsvDataReader('project', 'data/NEM/projects.csv')
projects = projects_cdr.read()

borehole_cdr = CsvDataReader('borehole', 'data/NEM/boreholes.csv')
boreholes = borehole_cdr.read()

parameter_cdr = CsvDataReader('parameter', 'data/NEM/LabData.Parameters.csv')
parameters = parameter_cdr.read()

print "Record counts: "
print "  projects:   " + str(len(projects))
print "  boreholes:  " + str(len(boreholes))
print "  parameters: " + str(len(parameters))
print "TOTAL: "+str(len(projects)+len(boreholes)+len(parameters))
print "\nExample JSON data: project[0]"
print json.dumps(projects[0], indent=2)

print "\nExample JSON data: borehole[0]"
print json.dumps(boreholes[0], indent=2)

print "\nExample JSON data: parameters[77]"
print json.dumps(parameters[77], indent=2)

Record counts: 
  projects:   4
  boreholes:  16
  parameters: 78
TOTAL: 98

Example JSON data: project[0]
{
  "id": "project-20170622-174516-52956332", 
  "project_coordinate_system": "UTM", 
  "type": "project", 
  "project_code": "NEM", 
  "name": "Nam E-Moun"
}

Example JSON data: borehole[0]
{
  "borehole_code": "NE1-01", 
  "collar_elevation_masl": "", 
  "y_project_coordinate": "1720999.33", 
  "x_project_coordinate": "717190.06", 
  "type": "borehole", 
  "id": "borehole-20170622-174516-39266317"
}

Example JSON data: parameters[78]
{
  "sample_description": "dry", 
  "measurement_convention": "Es", 
  "test_standard": "ASTM 2166", 
  "sample_code": "NE08", 
  "test_parameters": [
    {
      "units": "", 
      "description": "", 
      "value": ""
    }, 
    {
      "units": "", 
      "description": "", 
      "value": ""
    }
  ], 
  "laboratory": "LTEC", 
  "test_comment": "", 
  "id": "parameter-20170622-174516-24266828", 
  "laboratory_test_code": "32", 
  "test_type":

# Store in Couchbase

 * Only run this once!
 * Then comment out the *cb.insert* statement

In [21]:
from couchbase.bucket import Bucket
cb = Bucket('couchbase://localhost/geotechnical')

def load_records(records):
    for record in records:
        id = record['id']
        ##cb.upsert(id,record)  # DISABLED!!!
        print "record: "+id+" ..inserted"

print "\nLoading: projects"
load_records(projects)

print "\nLoading: boreholes"
load_records(boreholes)

print "\nLoading: parameters"
load_records(parameters)


Loading: projects
record: project-20170622-174516-52956332 ..inserted
record: project-20170622-174516-76927550 ..inserted
record: project-20170622-174516-72672078 ..inserted
record: project-20170622-174516-10551702 ..inserted

Loading: boreholes
record: borehole-20170622-174516-39266317 ..inserted
record: borehole-20170622-174516-36599884 ..inserted
record: borehole-20170622-174516-66318427 ..inserted
record: borehole-20170622-174516-66574135 ..inserted
record: borehole-20170622-174516-05582985 ..inserted
record: borehole-20170622-174516-37573956 ..inserted
record: borehole-20170622-174516-27653986 ..inserted
record: borehole-20170622-174516-23912291 ..inserted
record: borehole-20170622-174516-85469632 ..inserted
record: borehole-20170622-174516-32247481 ..inserted
record: borehole-20170622-174516-94840249 ..inserted
record: borehole-20170622-174516-60302042 ..inserted
record: borehole-20170622-174516-11300620 ..inserted
record: borehole-20170622-174516-48544201 ..inserted
record: bor

# Read Some Data

## Create Default Index

 * Required for querying. If you don't do this you get an error when you run queries
 * This should only be run once. Note yet sure how to ensure that happend
 * For now, run it once and then comment out the *cb.n1ql_query* statement

In [24]:
from couchbase.bucket import Bucket

cb = Bucket('couchbase://localhost/geotechnical')
##cb.n1ql_query('CREATE PRIMARY INDEX ON geotechnical').execute()  # TODO this should only be run one time!
# running above query gave HTTPError: ... http_status=500. Primry Index was however created (couchbase browser).

In [25]:
import json

from DataOrganizer import CouchbaseSelector

# -- Open the database

db = CouchbaseSelector('localhost','geotechnical')

fields = ['parameter', 'type', 'value', 'value_unit']
where = {'type': 'parameter', 'parameter': 'E','data_source': 'Laboratory'}

# db.getParameter(parameter_name, fields, where)

recs = []
for rec in db.select(None, None):
#for rec in db.select(fields, where):
    #print json.dumps(rec, indent=2)
    recs.append(rec)
    
print json.dumps(recs[0],indent=2)
print "\nTotal records: "+str(len(recs))

{
  "geotechnical": {
    "borehole_code": "NE1-05", 
    "collar_elevation_masl": "", 
    "y_project_coordinate": "1721741.8", 
    "x_project_coordinate": "712184.8", 
    "type": "borehole", 
    "id": "borehole-20170622-174516-05582985"
  }
}

Total records: 98
